In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [ ]:
# 2d array filled with -1 values
grid = np.full((10,10), -1)
lane = grid[::-1, 3]
lane[0] = 0

print(grid)

In [ ]:
def create_coordinates(v_idxs, h_idxs):
    # Use numpy's meshgrid function to generate coordinate matrices
    y, x = np.meshgrid(v_idxs, h_idxs)

    # Stack the x and y coordinate arrays together into one 2D array
    # The new shape of coordinates will be (2, len(v_idxs) * len(h_idxs))
    # Then transpose it to get shape (len(v_idxs) * len(h_idxs), 2)
    coordinates = np.vstack([x.ravel(), y.ravel()]).T

    return coordinates


v_idxs = np.array([1, 2, 3])
h_idxs = np.array([4, 5, 6])
coordinates = create_coordinates(v_idxs, h_idxs)

print(coordinates)


In [ ]:
grid = np.full((10,10), -1)
for i in range(10):
    grid[i, 3] = 0
    if random.random() < 0.3:
        grid[i, 3] = 1
    if random.random() < 0.3:
        grid[i, 3] = 2
    if random.random() < 0.3:
        grid[i, 3] = 3

# print(grid)

def TestLane(sim, subarray):
    mygrid = sim
    myroad = subarray
    mycars = subarray.copy()
    mytrafficlights = subarray.copy()
    mycars[mycars > 1] = 0
    mytrafficlights[mytrafficlights <= 1] = -1
    mytrafficlights[mytrafficlights == 2] = 0
    mytrafficlights[mytrafficlights == 3] = 1


    print(f'grid:\n{mygrid}', end='\n\n')
    print(f'road:\n{myroad}', end='\n\n')
    print(f'cars:\n{mycars}', end='\n\n')
    print(f'traffic lights:\n{mytrafficlights}', end='\n\n')

TestLane(grid, grid[::, 3])

In [ ]:
import numpy as np
import random
from IPython.display import clear_output
from time import sleep

class TestLane:
    """A class to test the lane class. Has a length, a road, cars, and traffic lights."""
    def __init__(self, road, entry_rate=0.1):
        self.road = road
        self.next_road = np.zeros_like(road)
        self.length = len(road)
        self.cars = []
        self.traffic_lights = []
        self.entry_rate = entry_rate

    def update(self):
        for light in self.traffic_lights:
            light.update()
        for car in self.cars:
            car.move()

        if random.random() < self.entry_rate and self.next_road[0] == 0:
            self.add_car()
        self.road = self.next_road.copy()
        self.next_road = np.zeros_like(self.road)

    def add_light(self, position, duration=10):
        self.traffic_lights.append(TestTrafficLight(self, position, duration))
        self.road[position] = 3

    def add_car(self):
        self.cars.append(TestCar(self, 0))
        self.next_road[0] = 2

    def __repr__(self):
        output = ''
        for i in range(self.length):
            if self.road[i] == 1:
                # black square
                output += '⬛'
            elif self.road[i] == 2:
                # blue car
                output += '🚙'
            elif self.road[i] == 3:
                # red circle
                output += '🔴'
            elif self.road[i] == 4:
                # green circle
                output += '🟢'
            else:
                # white square
                output += '⬜'
        return output

class TestTrafficLight:
    """A class to test the traffic light class. Has a position, a state, and a timer."""
    def __init__(self, lane, position, duration):
        self.lane = lane
        self.position = position
        self.state = 0
        self.timer = 0
        self.duration = duration

    def update(self):
        if self.state == 0:
            self.timer += 1
            if self.timer == self.duration:
                self.state = 1
                self.timer = 0
        else:
            self.timer += 1
            if self.timer == self.duration:
                self.state = 0
                self.timer = 0
        self.lane.next_road[self.position] = self.state + 3
    
    def __repr__(self):
        return f'TrafficLight(position={self.position}, state={self.state})'
    

class TestCar:
    """A class to test the car class. Has a max_speed, a speed, a position."""
    def __init__(self, lane, position, max_speed=3):
        self.lane = lane
        self.position = position
        self.max_speed = max_speed
        self.speed = 0
    
    def look_ahead(self):
        for i in range(self.position + 1, self.lane.length):
            if self.lane.road[i] in [2,3]:
                return i - self.position
        return self.lane.length - self.position

    def move(self):
        dist = self.look_ahead()
        # Check for red light one position ahead, ignore if at end of road
        if self.position + 1 < self.lane.length and self.lane.road[self.position + 1] in [2,3]:
            self.speed = 0
        elif dist <= self.speed:
            self.speed = dist - 1
        else:
            self.speed = min(self.speed + 1, self.max_speed)
        self.position += self.speed
        if self.position >= self.lane.length:
            self.lane.cars.remove(self)
        else:
            self.lane.next_road[self.position] = 2

    def __repr__(self):
        return f'Car(position={self.position}, speed={self.speed})'

lane = TestLane(np.zeros(20))
lane.add_light(9, 30)

for i in range(100):
    print(lane)
    lane.update()
    sleep(0.1)
    clear_output(wait=True)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

history = np.zeros((20, 20))
history[0, 0] = 1
history[0, 1] = 2
history[0, 2] = 3
history[0, 3] = 4
history[1, 0] = 5


cmap = ListedColormap(['white', 'black', 'blue', 'red', 'green'])
v_idxs = np.array([6, 12])
h_idxs = np.array([4, 10])
v_directions = ['N', 'S']
h_directions = ['E', 'W']

fig = plt.figure()
ax = plt.gca()

im = ax.imshow(history, cmap=cmap, interpolation='none')

ax.set_xticks(v_idxs)
ax.set_xticklabels(['⬆' if _ == 'N' else '⬇' for _ in v_directions ])
ax.set_yticks(h_idxs)
ax.set_yticklabels(['➡' if _ == 'E' else '⬅' for _ in h_directions ])

# Set ticks on both sides of axes on
ax.tick_params(right= True, top=True, labelright=True, labeltop=True)

ax.set_title("2D Traffic Simulation")
fig.tight_layout()
plt.show()


In [47]:
class Lane:
    def __init__(self):
        self.cars = []

class Car:
    def __init__(self, speed):
        self.speed = speed

car = Car(5)  # create a Car object
lane = Lane()  # create a Lane object
lane.cars.append(car)  # add the Car object to the Lane object
d = {'car': car}  # put it in a dictionary

car.speed = 10  # modify the Car object

print(id(d['car']))  # outputs: 10

d['car'].speed = 20  # modify the Car object again

print(id(car))  # outputs: 20

lane.cars[0].speed = 30  # modify the Car object again

print(id(car))  # outputs: 30


140288713701408
140288713701408
140288713701408
